# < 첨부하고자 하는 코드 >

## [1].출고 vs 배송 지연 일수 계산 (독립성 판단)

In [ ]:
merged['shipping_delay_days'] = (merged['order_delivered_carrier_date'] - merged['order_approved_at']).dt.days
merged['delivery_delay_days'] = (merged['order_delivered_customer_date'] - merged['order_delivered_carrier_date']).dt.days


➡ 단계별 지연을 분리해서 각각 분석 \



## [2].지연 여부 판단 (이진값으로 변환후 가공)

In [ ]:
merged['is_shipping_late'] = merged['order_delivered_carrier_date'] > merged['shipping_limit_date']
merged['is_delivery_late'] = merged['order_delivered_customer_date'] > merged['order_estimated_delivery_date']


➡ 지연 판단을 정량적 기준으로 명확화 \



## [3]. 카이제곱 검정 ( 출고, 배송 지연 독립성 검정)

In [ ]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(crosstab)


➡ 출고/배송 지연 간 독립성 여부를 객관적으로 검정 (통계적 검증)



## [4].신뢰도 지수 생성 및 셀러 신뢰도 기반 분석



In [ ]:
# 셀러별 신뢰도 지수 추출
seller_delay_df['trust_score'] = (1 - seller_delay_df['shipping_delay_rate']) * (1 - seller_delay_df['delivery_delay_rate'])
# 지역별 신뢰도 평균 추출
region_grouped['avg_trust_score'] = 1 - (region_grouped['avg_shipping_delay'] + region_grouped['avg_delivery_delay'])


➡ 지연률을 활용한 비즈니스 KPI 도출 (신뢰도 지수) \


## [5].신뢰도와 리뷰 점수 상관관계 분석

In [ ]:
from scipy.stats import pearsonr
corr, p_value = pearsonr(seller_grouped['is_shipping_late'], seller_grouped['is_delivery_late'])

corr, p_value = pearsonr(seller_reviews['avg_review_score'], seller_reviews['trust_score'])


➡ 고객 만족과 물류 신뢰도 사이의 실질적인 연결고리 도출 \



## [6].지역 집계 및 시각화용 가공

In [ ]:
region_grouped = seller_grouped.groupby('seller_city').agg({
    'is_shipping_late': 'mean',
    'is_delivery_late': 'mean',
    'trust_score': 'mean',
    'seller_id': 'count'
}).reset_index()
region_grouped.columns = ['seller_city', 'avg_shipping_delay', 'avg_delivery_delay', 'avg_trust_score', 'seller_count']


➡ 지역 단위 전략을 위해 요약 통계로 변환

##[7].신뢰도 기준 시각화 설계 (상위/하위 그룹 비교)


In [ ]:
sns.scatterplot(data=top_bottom, x='avg_trust_score', y='seller_count', hue='seller_city')

➡ 단순히 값만 보는 게 아니라, 전략적 입지 선정까지 이어지는 흐름 설계

# * 위 코드의 column 의미 및 설명

 1. shipping_delay_days \
의미: 주문 승인부터 판매자 출고까지 걸린 시간 (출고 지연 확인용) \
생성 방식: order_delivered_carrier_date - order_approved_at 계산해서 만든 파생 컬럼

 2. delivery_delay_days \
의미: 출고 후 고객에게 도착하기까지 걸린 시간 (배송 지연 확인용) \
생성 방식: order_delivered_customer_date - order_delivered_carrier_date 계산해서 만든 파생 컬럼

 3. is_shipping_late \
의미: 판매자가 출고 마감기한(shipping_limit_date)을 넘겼는지 여부 \
생성 방식: order_delivered_carrier_date > shipping_limit_date 조건으로 True/False 생성

 4. is_delivery_late \
의미: 고객에게 배송이 예상일보다 늦었는지 여부 \
생성 방식: order_delivered_customer_date > order_estimated_delivery_date 조건으로 True/False 생성

 5. trust_score \
의미: 출고 및 배송 지연이 없을수록 높은 신뢰도를 갖는 셀러 지수 (0~1) \
생성 방식: 셀러별로 1 - (출고 지연률 + 배송 지연률) 계산

 6. avg_review_score \
의미: 고객이 셀러에게 남긴 평균 별점 (만족도 지표) \
생성 방식: reviews 데이터셋과 orders 병합 후, 셀러 기준으로 평균 별점 집계

 7. seller_count \
의미: 특정 도시에 존재하는 셀러의 수 \
생성 방식: seller_id를 seller_city 기준으로 groupby 후 count

 8. seller_city \
의미: 셀러의 지역/도시 정보 \
생성 방식: sellers 데이터셋에서 가져와 order_items 또는 집계 데이터에 병합